In [1]:
import qiskit
import numpy as np

%matplotlib inline
import matplotlib.pyplot as plt

ModuleNotFoundError: No module named 'matplotlib'

# Main functions

In [2]:
def swap_qubits(qc, qreg):
    """
    Reverse the order of qubits
    Note: qiskit qubit order is most significant last (i.e. q[n-1])
    """
    for i in range(len(qreg)//2):
        qc.swap(qreg[i], qreg[-i-1])

def qft(qc, qreg, do_swaps=False, inverse=False):
    """Circuit for QFT. Assumes most significant bit last."""
    
    if inverse:
        sign = -1
    else:
        qreg = qreg[::-1]
        sign = 1
        
    # swap qubits -- inverse QFT
    if do_swaps and inverse:
        swap_qubits(qc, qreg)
    
    # do QFT
    for i, target in enumerate(qreg):
        qc.h(target)
        for k, control in enumerate(qreg[i+1:]):
            qc.cu1(sign*np.pi/2**(k+1), control, target)
            
    # swap qubits -- forward QFT
    if do_swaps and not inverse:
        swap_qubits(qc, qreg)

def qiskit_qft(qc, qreg, do_swaps=False, inverse=False):
    """Qiskit circuit library for QFT"""
    # Qiskit uses most significant bit last convention
    # However, the QFT in the circuit library uses most significant bit first
    # therefore we need to swap the qubits
    inst = qiskit.circuit.library.QFT(num_qubits=len(qreg), do_swaps=do_swaps, inverse=inverse)
    qc.append(inst, qreg[::-1])


# Circuit

In [3]:
# inputs
n = 5

# quantum circuit
q = qiskit.QuantumRegister(n, name='q')
qc = qiskit.QuantumCircuit(q, name='qc')

qft(qc, q, inverse=False)

qc.draw()
# print(qc.qasm())

»
q_0: ───────────────────────────────────────■───────────────────■────────────»
                                            │                   │            »
q_1: ─────────────────────────■─────────────┼───────■───────────┼──────■─────»
                              │             │       │     ┌───┐ │      │pi/2 »
q_2: ─────────────■───────────┼──────■──────┼───────┼─────┤ H ├─┼──────■─────»
                  │     ┌───┐ │      │pi/2  │       │pi/4 └───┘ │pi/8        »
q_3: ──────■──────┼─────┤ H ├─┼──────■──────┼───────■───────────■────────────»
     ┌───┐ │pi/2  │pi/4 └───┘ │pi/8         │pi/16                           »
q_4: ┤ H ├─■──────■───────────■─────────────■────────────────────────────────»
     └───┘                                                                   »
«                        ┌───┐
«q_0: ─■───────────■─────┤ H ├
«      │     ┌───┐ │pi/2 └───┘
«q_1: ─┼─────┤ H ├─■──────────
«      │pi/4 └───┘            
«q_2: ─■──────────────────────
«                             
«q_3: ────────────────────────
«                             
«q_4: ────────────────────────
«

In [4]:
# inputs
n = 5

# quantum circuit
q = qiskit.QuantumRegister(n, name='q')
qc = qiskit.QuantumCircuit(q, name='qc')

qiskit_qft(qc, q, inverse=False)

qc.decompose().draw()

»
q_0: ───────────────────────────────────────────────────■──────────────■─────»
                                                        │              │     »
q_1: ────────────────────────────────■───────────■──────┼───────■──────┼─────»
                                     │     ┌───┐ │      │       │pi/2  │     »
q_2: ──────────────────■──────■──────┼─────┤ H ├─┼──────┼───────■──────┼─────»
                 ┌───┐ │      │pi/2  │     └───┘ │pi/4  │              │pi/8 »
q_3: ──────■─────┤ H ├─┼──────■──────┼───────────■──────┼──────────────■─────»
     ┌───┐ │pi/2 └───┘ │pi/4         │pi/8              │pi/16               »
q_4: ┤ H ├─■───────────■─────────────■──────────────────■────────────────────»
     └───┘                                                                   »
«                        ┌───┐
«q_0: ──────■──────■─────┤ H ├
«     ┌───┐ │      │pi/2 └───┘
«q_1: ┤ H ├─┼──────■──────────
«     └───┘ │pi/4             
«q_2: ──────■─────────────────
«                             
«q_3: ────────────────────────
«                             
«q_4: ────────────────────────
«

# Compare with FFT

## Functions

In [5]:
# Qiskit simulator
def qiskit_get_statevector(circuit):
    backend = qiskit.Aer.get_backend('statevector_simulator')
    job = qiskit.execute(circuit, backend)
    return job.result().get_statevector()

# QFT reorder
def get_basis_states(n_qubits):
    return ['{{:0{:d}b}}'.format(n_qubits).format(x) for x in range(2**n_qubits)]

def qft_get_reorder_idx(n_qubits):
    """Get indices for reordering QFT basis states"""
    return [int(state[::-1], 2) for state in get_basis_states(n_qubits)]

def qft_get_reorder_states(n_qubits):
    """Get reordered basis states in QFT outputs"""
    basis_states = get_basis_states(n_qubits)
    return [basis_states[int(state[::-1], 2)] for state in basis_states]

def reorder_qft(a, axis=0):
    """Reorder QFT outputs by reversing qubits"""
    n_qubits = int(np.round(np.log2(len(a))))
    return np.asarray(a).take(qft_get_reorder_idx(n_qubits), axis=axis)

# Do FFT
def fft_by_qft(x, inverse=False, use_classical_swaps=False):
    # QFT = inverse classical DFT
    inverse = not inverse
    # number of qubits
    n = int(np.log2(len(x)))
    assert len(x) == 2**n
    # construct circuit
    qreg = qiskit.QuantumRegister(n)
    qc = qiskit.QuantumCircuit(qreg)
    # initialize circuit with input state vector
    qc.initialize(x, qreg)
    # reverse qubit order classically -- inverse QFT
    if use_classical_swaps and inverse:
        qreg = qreg[::-1]
    # apply QFT
    qft(qc, qreg, do_swaps=not use_classical_swaps, inverse=inverse)
    # execute circuit
    result = qiskit_get_statevector(qc)
    # reverse qubit order classically -- forward QFT
    if use_classical_swaps:
        result = reorder_qft(result)
    return result

def direct_dft(x, inverse=False):
    n = len(x)
    sign = 1 if inverse else -1
    gen = (sum(x[i]*np.exp(sign*2j*np.pi*k*i/n)/np.sqrt(n) for i in range(n)) for k in range(n))
    return np.fromiter(gen, np.complex, count=n)

# Normalization
def normalize(x):
    x = np.asarray(x)
    return x / np.linalg.norm(x)


## Circuit

In [6]:
# x = [1, 2, 3, 4]
x = [1, 2-1j, -1j, -1+2j]
x = normalize(x)

print("Input: %s" % x)

print("------- Forwrad FFT -------")
dft = direct_dft(x)
np_fft = np.fft.fft(x)
qft_fft = fft_by_qft(x)
qft_fft_c = fft_by_qft(x, use_classical_swaps=True)

np_fft = np_fft / np.sqrt(len(x))

print(dft)
print(np_fft)
print(qft_fft)
print(qft_fft_c)
print(np.allclose(np_fft, dft))
print(np.allclose(qft_fft, dft))
print(np.allclose(qft_fft_c, dft))

print("------- Inverse FFT -------")
idft = direct_dft(dft, inverse=True)
np_ifft = np.fft.ifft(np_fft)
qft_ifft = fft_by_qft(qft_fft, inverse=True)
qft_ifft_c = fft_by_qft(qft_fft_c, inverse=True, use_classical_swaps=True)

np_ifft = np_ifft * np.sqrt(len(np_fft))

print(idft)
print(np.allclose(idft, x))

print(np_ifft)
print(np.allclose(np_ifft, x))

print(qft_ifft)
print(np.allclose(qft_ifft, x))

print(qft_ifft_c)
print(np.allclose(qft_ifft_c, x))

Input: [ 0.28867513+0.j          0.57735027-0.28867513j -0.        -0.28867513j
 -0.28867513+0.57735027j]
------- Forwrad FFT -------
[ 2.88675135e-01+0.j         -2.88675135e-01-0.28867513j
  1.38777878e-16-0.28867513j  5.77350269e-01+0.57735027j]
[ 0.28867513+0.j         -0.28867513-0.28867513j  0.        -0.28867513j
  0.57735027+0.57735027j]
[ 2.88675135e-01-8.32667268e-17j -2.88675135e-01-2.88675135e-01j
 -1.00942981e-16-2.88675135e-01j  5.77350269e-01+5.77350269e-01j]
[ 2.88675135e-01-8.32667268e-17j -2.88675135e-01-2.88675135e-01j
 -1.00942981e-16-2.88675135e-01j  5.77350269e-01+5.77350269e-01j]
True
True
True
------- Inverse FFT -------
[ 2.88675135e-01+1.11022302e-16j  5.77350269e-01-2.88675135e-01j
  5.55111512e-17-2.88675135e-01j -2.88675135e-01+5.77350269e-01j]
True
[ 2.88675135e-01-2.77555756e-17j  5.77350269e-01-2.88675135e-01j
  2.77555756e-17-2.88675135e-01j -2.88675135e-01+5.77350269e-01j]
True
[ 2.88675135e-01-2.77555756e-17j  5.77350269e-01-2.88675135e-01j
 -7.318740

# QFT unitary matrix

In [7]:
def qiskit_get_unitary(circuit):
    backend = qiskit.BasicAer.get_backend('unitary_simulator')
    job = qiskit.execute(circuit, backend)
    return job.result().get_unitary()


## Using swaps on circuit

In [8]:
n = 5

def qiskit_qft_unitary(n, do_swaps=False, inverse=False):
    qc = qiskit.QuantumCircuit(n)
    qiskit_qft(qc, range(n), do_swaps=do_swaps, inverse=inverse)
    return qiskit_get_unitary(qc)

fwd_swaps = qiskit_qft_unitary(n, do_swaps=True)
fwd_no_swaps = qiskit_qft_unitary(n, do_swaps=False)

inv_swaps = qiskit_qft_unitary(n, do_swaps=True, inverse=True)
inv_no_swaps = qiskit_qft_unitary(n, do_swaps=False, inverse=True)

print(np.allclose(fwd_swaps, reorder_qft(fwd_no_swaps)))
print(np.allclose(inv_swaps, reorder_qft(inv_no_swaps, axis=1)))

True
True


## Using swaps off circuit

In [9]:
n = 2

def qiskit_qft_unitary(n, do_swaps=False, inverse=False, use_classical_swaps=False):
    qc = qiskit.QuantumCircuit(n)
    qreg = range(n)
    if do_swaps and use_classical_swaps and inverse:
        qreg = qreg[::-1]
    qiskit_qft(qc, qreg, do_swaps=do_swaps and not use_classical_swaps, inverse=inverse)
    unitary = qiskit_get_unitary(qc)
    if do_swaps and use_classical_swaps:
        unitary = reorder_qft(unitary)
    return unitary
        

fwd_swaps = qiskit_qft_unitary(n, do_swaps=True, use_classical_swaps=False)
fwd_no_swaps = qiskit_qft_unitary(n, do_swaps=True, use_classical_swaps=True)

inv_swaps = qiskit_qft_unitary(n, do_swaps=True, inverse=True, use_classical_swaps=False)
inv_no_swaps = qiskit_qft_unitary(n, do_swaps=True, inverse=True, use_classical_swaps=True)

print(np.round(fwd_swaps, 15))

print(np.allclose(fwd_swaps, fwd_no_swaps))
print(np.allclose(inv_swaps, inv_no_swaps))

[[ 0.5+0.j   0.5-0.j   0.5-0.j   0.5-0.j ]
 [ 0.5+0.j   0. +0.5j -0.5+0.j  -0. -0.5j]
 [ 0.5+0.j  -0.5+0.j   0.5-0.j  -0.5+0.j ]
 [ 0.5+0.j  -0. -0.5j -0.5+0.j   0. +0.5j]]
True
True


# QFT unitary matrix ($n=2$)

In [10]:
# Forward transform matrix
h0 = np.matrix([[1, 0, 1, 0], [0, 1, 0, 1], [1, 0, -1, 0], [0, 1, 0, -1]])/np.sqrt(2)
h1 = np.matrix([[1, 1, 0, 0], [1, -1, 0, 0], [0, 0, 1, 1], [0, 0, 1, -1]])/np.sqrt(2)
cr2 = np.matrix([[1, 0, 0, 0], [0, 1, 0, 0], [0, 0, 1, 0], [0, 0, 0, 1j]])
u = h1*cr2*h0
u = np.matrix(reorder_qft(u))
u

matrix([[ 0.5+0.j ,  0.5+0.j ,  0.5+0.j ,  0.5+0.j ],
        [ 0.5+0.j ,  0. +0.5j, -0.5+0.j ,  0. -0.5j],
        [ 0.5+0.j , -0.5+0.j ,  0.5+0.j , -0.5+0.j ],
        [ 0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.5j]])

In [11]:
# Forward transform matrix
qc = qiskit.QuantumCircuit(2)
qc.h(1)
qc.cu1(np.pi/2, 0, 1)
qc.h(0)
result = qiskit_get_unitary(qc)
result = np.round(result, 15)
result = np.matrix(reorder_qft(result))
result

matrix([[ 0.5+0.j ,  0.5-0.j ,  0.5-0.j ,  0.5-0.j ],
        [ 0.5+0.j ,  0. +0.5j, -0.5+0.j , -0. -0.5j],
        [ 0.5+0.j , -0.5+0.j ,  0.5-0.j , -0.5+0.j ],
        [ 0.5+0.j , -0. -0.5j, -0.5+0.j ,  0. +0.5j]])

In [12]:
# Forward transform matrix
result = qiskit_qft_unitary(2, do_swaps=True, inverse=True)
result = np.round(result, 15)
# result = reorder_qft(result)
result

array([[ 0.5+0.j ,  0.5-0.j ,  0.5-0.j ,  0.5-0.j ],
       [ 0.5+0.j ,  0. -0.5j, -0.5+0.j ,  0. +0.5j],
       [ 0.5+0.j , -0.5+0.j ,  0.5-0.j , -0.5+0.j ],
       [ 0.5+0.j , -0. +0.5j, -0.5+0.j , -0. -0.5j]])